In [1]:
%%capture
!pip3 install google-cloud-aiplatform==1.0.0 --upgrade
!pip3 install kfp google-cloud-pipeline-components==0.1.1 --upgrade
!pip3 install scikit-learn
!pip3 install pandas

In [2]:
import uuid
from kfp import dsl
import kfp
from google.cloud import aiplatform
from kfp.v2.dsl import component
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, component, ClassificationMetrics, Metrics)
from google_cloud_pipeline_components import aiplatform as gcc_aip
from typing import NamedTuple

In [3]:
#https://stackoverflow.com/a/54028874
%load_ext dotenv
%dotenv

import os
PROJECT_ID = os.environ['PROJECT_ID']
BUCKET_NAME = os.environ['BUCKET']

PIPELINE_ROOT = 'gs://{}/pipeline_root'.format(BUCKET_NAME)
REGION = 'us-central1'

print(PROJECT_ID)
print(BUCKET_NAME)
print(PIPELINE_ROOT)

kubeflow-demos
user-group-demo
gs://user-group-demo/pipeline_root


In [18]:
@component(packages_to_install=["google-cloud-aiplatform"])
def run_batch_job():

    from typing import Sequence, Union
    from google.cloud import aiplatform, aiplatform_v1

    def create_batch_prediction_job_dedicated_resources_sample(
        project: str,
        location: str,
        model_resource_name: str,
        job_display_name: str,
        gcs_source: Union[str, Sequence[str]],
        gcs_destination: str,
        machine_type: str = "n1-standard-2",
        accelerator_count: int = 1,
        accelerator_type: Union[str, aiplatform_v1.AcceleratorType] = "NVIDIA_TESLA_K80",
        starting_replica_count: int = 1,
        max_replica_count: int = 1,
        sync: bool = True,
    ):
        aiplatform.init(project=project, location=location)

        my_model = aiplatform.Model(model_resource_name)

        batch_prediction_job = my_model.batch_predict(
            job_display_name=job_display_name,
            gcs_source=gcs_source,
            gcs_destination_prefix=gcs_destination,
            machine_type=machine_type,
            accelerator_count=accelerator_count,
            accelerator_type=accelerator_type,
            starting_replica_count=starting_replica_count,
            max_replica_count=max_replica_count,
            sync=sync,
        )

        batch_prediction_job.wait()

        print(batch_prediction_job.display_name)
        print(batch_prediction_job.resource_name)
        print(batch_prediction_job.state)
        return batch_prediction_job

        
    create_batch_prediction_job_dedicated_resources_sample(
            model_resource_name="4789441864266678272",
            job_display_name="test3",
            gcs_source="gs://test-fast/batch_test.csv",
            gcs_destination="gs://test-fast",
            project="kubeflow-demos",
            location="us-central1",
        )

In [19]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [20]:
@kfp.dsl.pipeline(name="batch-test" + str(uuid.uuid4()))
def pipeline(
    project: str = PROJECT_ID,
    bucket: str = BUCKET_NAME,
    model_id: str = "4789441864266678272"
):
    batch_task = run_batch_job()

In [21]:
from kfp.v2 import compiler

compiler.Compiler().compile(pipeline_func=pipeline, 
                            package_path="dag-"+TIMESTAMP+".json")

/Users/yarkoni/projects/ds-churn/venv/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1263: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [22]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

/Users/yarkoni/projects/ds-churn/venv/lib/python3.9/site-packages/kfp/v2/google/client/client.py:169: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  warnings.warn(


In [23]:
response = api_client.create_run_from_job_spec(
    "dag-"+TIMESTAMP+".json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"model_id": "4789441864266678272"},
)